# Использование C++ в python

С помощью модуля `ctypes` возможно подгружать символы из динамической C-библиотеки в виде функций python.
Функции в библиотеки должны иметь интерфейс языка C, а не C++ (проблема манглинга имён).
Сам код при этом можент быть написан на языке C++.

Для интерфейса будем использовать указатели для массивов и дополнительный параметр количества элементов массива.
Выделения памяти под указатели будет осуществлять python.
Массивы будут использоваться как входной и выходной параметр, т.е. сами функции не должно что-либо возвращать.

Файл `dft.cpp` находится в корне проекта и может быть использован внутри ipynb-файла.

_Замечание_: Если файл не загружается вместе с ipynb-файлом, его можно встроить с помощью директивы `%%wfitefile`.
Для этого создайте новую ячейку и поместите в неё содержимое файла `dft.cpp`.
В самом верху ячейки укажите `%%writefile dft.cpp`:
```python
%%writefile dft.cpp
#include <algorithm>
#include <cmath>
#include <complex>
...
```
Таким образом на текущей файловой системе будет создан файл `dft.cpp`.

## Компиляция

С помощью директивы `%%bash` можно писать инструкции для компиляции полученного файла `dft.cpp` в динамическую библиотеку `libdft.so`.
После этих операций библиотека `libdft.so` загружается в окружение python с помощью вызова `CDLL`.

_Замечание_: данный ipynb-файл может быть запущен на операционной системе Linux, где доступен компилятор `g++`, а также в colab.research.google.com и github codespace.
Компилируем файл `dft.cpp` в динамическую библиотеку `libdft.so` в режиме полной оптимизации (`-O2`).

In [1]:
%%bash
g++ -std=c++23 -O2 -pedantic -fPIC -shared -o libdft.so dft.cpp
ls -l .

total 52
-rw-rw-rw- 1 vscode root     348 Apr 25 16:54 CMakeLists.txt
-rw-rw-rw- 1 vscode root    1070 Apr 25 16:54 LICENSE
-rw-rw-rw- 1 vscode root    1052 Apr 25 16:54 dft.cpp
-rw-rw-rw- 1 vscode root     630 Apr 25 16:54 dft.h
-rwxrwxrwx 1 vscode vscode 16424 Apr 25 17:06 libdft.so
-rw-rw-rw- 1 vscode root    1026 Apr 25 16:54 main.cpp
-rw-rw-rw- 1 vscode root    7835 Apr 25 17:06 pycxx.ipynb
-rw-rw-rw- 1 vscode root      32 Apr 25 16:54 requirements.txt


## Вызов C-функций

Для вызова C-функций можно подготовить специальный класс, который содержит в себе управление ресурсами
библиотеки и перенаправление вызовов интерфейса.

In [2]:
import ctypes as ct
import numpy as np
from timeit import timeit

class MyCxxLib:
    complex128 = ct.POINTER(2*ct.c_double)

    def __init__(self, name='./libdft.so'):
        self.libdft = ct.CDLL(name)

    def dft(self, x, out):
        self.libdft.dft(ct.c_size_t(len(x)), x.ctypes.data_as(MyCxxLib.complex128), out.ctypes.data_as(MyCxxLib.complex128))

    def __del__(self):
        libdl = ct.CDLL(None)
        handle = self.libdft._handle
        del self.libdft
        libdl.dlclose(handle)

if 'mylib' in locals():
    del mylib

mylib = MyCxxLib()

# Эксперименты

Сгенерируем данные случайным образом и проведём сравнение со стандартной функией `np.fft.fft`.
Для определения ошибки используется Евлкидова норма.
Для вычисления времени вычисления используется модуль `timeit`, которому можно задать количество повторений.
Полученное время затем делится на количество повторений, чтобы получить среднее время работы алгоритма.
Такой подход позволяет устранить "всплески" на временной шкале, вызванные многозадачностью вычислительной машины (переключение между потоками и процессами), влиянием кешей процессора и обращениями к дискам.

_Замечание_: все данные генерируются с помощью одного объекта, который был получен из фиксированного источника (seed), равного 29.
Это гарантирует, что при любых перезапусках будут **всегда** получатся одни и те же числа (фиксирование генератора случайных чисел).

In [4]:
rng = np.random.default_rng(29)
N = 10
xr = rng.uniform(-100, 100, size=N)
xi = rng.uniform(-100, 100, size=N)
x = xr + 1j * xi
y = np.zeros(N, dtype=np.complex128)

t = timeit(lambda: mylib.dft(x, y), number=1000) / 1000
print("Time:", t, 'sec')

y2 = np.fft.fft(x) / np.sqrt(N)
print("Error:", np.linalg.norm(y2 - y))

Time: 1.1001802000009776e-05 sec
Error: 3.4075224359178426e-13
